# websearch

    '05_websearch.ipynb'    

# Tavily Wet Search

- LLM에게 웹 검색을 쥐어주자

In [5]:
# 모듈 세팅 
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [ ]:
# 강사님 모듈 세팅
from dotenv import load_dotenv
from pprint import pprint
load_dotenv() # .env 파일에 정의된 환경 변수들을 불러와서 현재 실행 중인 프로세스의 환경 변수로 로드하는 함수

True

In [7]:
from langchain_tavily import TavilySearch

search_tool = TavilySearch(
    max_results=5,
    topic="general",
    search_depth="basic",
    # time_range = "day"
    # include_domains = None
    # exclude_domains= None 
)

In [10]:
pprint(search_tool.invoke({'query' : '아 모르겠다'}))

{'answer': None,
 'follow_up_questions': None,
 'images': [],
 'query': '아 모르겠다',
 'request_id': '5967212e-de53-47c9-bbbd-4ad759653793',
 'response_time': 0.87,
 'results': [{'content': 'Play 아모르겠다 on SoundCloud and discover followers on '
                         'SoundCloud | Stream tracks, albums, playlists on '
                         'desktop and mobile.',
              'raw_content': None,
              'score': 0.6304352,
              'title': '아모르겠다 - SoundCloud',
              'url': 'https://soundcloud.com/xrahdpsxpvv5'},
             {'content': "'알겠다/모르겠다'는 영어 understand에 가깝고,. '안다/모른다'는 영어 know에 "
                         '가까운 것 같습니다. Q. 아시겠습니까? (Do you understand?) A. '
                         '알겠습니다. O',
              'raw_content': None,
              'score': 0.49553296,
              'title': '온라인가나다 상세보기(모르겠다 / 모른다) - 국립국어원',
              'url': 'https://m.korean.go.kr/front/onlineQna/onlineQnaView.do?mn_id=216&qna_seq=310083&pageIndex=1'},
             {'content

In [13]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough


llm = ChatOpenAI(model= 'gpt-4.1-nano')

prompt = PromptTemplate.from_template(
    template = """ 넌 웹 검색 결과를 요약해서 사용자 질문에 맞게 좋은 답을 해주는 챗봇이야

    질문: {query}
    검색결과: {search_results}
    질문에 맞는 답을 검색결과 바탕으로 간단하게 알려줘
    """
)




chain = (
    {
    'query' : RunnablePassthrough(), # 사용자 입력 그대로 이 자리에 들어옴
    'search_result' : RunnableLambda(
        lambda x : '\n'.join(
            [f'-{r['title']} ({r['url']})' for r in x['results']]
        )

        ) 
    }
    | prompt
    | llm 
    | StrOutputParser()
)


chain.invoke('강남 신세계에서 점심 먹기 좋은곳')

TypeError: string indices must be integers, not 'str'

In [ ]:
dump_chain = prompt | llm | StrOutputParser()
